<a href="https://colab.research.google.com/github/financieras/ai/blob/main/logistic_regression/src/sigmoide.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Logistic Regression

Pasos básicos para implementar la regresión logística "desde cero" usando el descenso del gradiente:

1. Preparación de los datos:

Separar la variable objetivo (House_Gryffindor) de las características
Agregar una columna de 1's para el término de sesgo (bias)
Separar los datos en conjuntos de entrenamiento y prueba


2. Función Sigmoide:

Necesitaremos implementar la función sigmoide: σ(z) = 1/(1 + e^(-z))
Esta función transformará nuestras predicciones lineales en probabilidades entre 0 y 1


3. Función de Pérdida:

Implementar la función de pérdida logarítmica (log loss)
Esta función mide qué tan bien están funcionando nuestras predicciones
Para regresión logística se usa la pérdida logarítmica binaria


4. Gradiente:

Calcular el gradiente de la función de pérdida con respecto a los pesos
Para regresión logística, el gradiente es: X^T * (h(X) - y) / m
Donde X son las características, h(X) son las predicciones, y son los valores reales, m es el número de muestras


5. Descenso del Gradiente:

Inicializar los pesos (pueden ser todos ceros o aleatorios)
Por cada iteración:

Calcular las predicciones usando la función sigmoide
Calcular el gradiente
Actualizar los pesos: w = w - α * gradiente
α es la tasa de aprendizaje (learning rate)

6. Criterio de Parada:

Definir un número máximo de iteraciones
O establecer un umbral mínimo de cambio en la función de pérdida
O ambos

7. Evaluación en datos de entrenamiento:

Hacer predicciones con los pesos optimizados
Convertir probabilidades a clases (0 o 1)
Calcular Accuracy en datos de entrenamiento


8. Evaluación en datos de test:

Cargar y preparar datos de test (añadir columna de 1's)
Hacer predicciones con los pesos optimizados
Calcular Accuracy en datos de test
Comparar Accuracy de train vs test para detectar posible sobreajuste

Partimos del archivo 'dataset_normalized_lite5.csv' que ya está normalizado y preparado.

El archivo está en Google Drive por lo que lo leemos y construimos un DataFrame.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd

# Ruta al archivo en Google Drive
ruta_archivo = '/content/drive/My Drive/dataset_normalized_lite5.csv'

# Leer el archivo CSV y crear el DataFrame
df = pd.read_csv(ruta_archivo)

# Eliminar las columnas especificadas
columns_to_drop = ['House_Hufflepuff', 'House_Ravenclaw', 'House_Slytherin']
df = df.drop(columns=columns_to_drop)

# Mostrar información sobre las columnas del DataFrame
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1277 entries, 0 to 1276
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Best Hand                      1277 non-null   float64
 1   Age                            1277 non-null   float64
 2   House_Gryffindor               1277 non-null   float64
 3   Herbology                      1277 non-null   float64
 4   Defense Against the Dark Arts  1277 non-null   float64
 5   Potions                        1277 non-null   float64
 6   Charms                         1277 non-null   float64
 7   Flying                         1277 non-null   float64
dtypes: float64(8)
memory usage: 79.9 KB
None


- Vamos a poner como primera columna House_Gryffindor.
- Vamos a mostrar las primeras filas del DataFrame.

In [3]:
from tabulate import tabulate

# Extraer la columna 'House_Gryffindor'
house_gryffindor = df.pop('House_Gryffindor')

# Insertar 'House_Gryffindor' como la primera columna
df.insert(0, 'House_Gryffindor', house_gryffindor)

# Mostrar las primeras filas del DataFrame en forma de tabla
print(tabulate(df.head(), headers='keys', tablefmt='fancy_grid'))

╒════╤════════════════════╤═════════════╤════════════╤═════════════╤═════════════════════════════════╤═══════════╤═══════════╤════════════╕
│    │   House_Gryffindor │   Best Hand │        Age │   Herbology │   Defense Against the Dark Arts │   Potions │    Charms │     Flying │
╞════╪════════════════════╪═════════════╪════════════╪═════════════╪═════════════════════════════════╪═══════════╪═══════════╪════════════╡
│  0 │                  0 │           0 │ -0.630834  │    0.866628 │                        1.0215   │ -0.702829 │  1.19791  │ -0.506096  │
├────┼────────────────────┼─────────────┼────────────┼─────────────┼─────────────────────────────────┼───────────┼───────────┼────────────┤
│  1 │                  0 │           1 │ -0.3078    │   -1.37602  │                        1.14449  │  0.412213 │ -1.01037  │ -1.39359   │
├────┼────────────────────┼─────────────┼────────────┼─────────────┼─────────────────────────────────┼───────────┼───────────┼────────────┤
│  2 │              

In [4]:
import numpy as np

# 1. Separar variable objetivo (y) de características (X)
y = df['House_Gryffindor']

# 2. Seleccionar las características (excluyendo House_Gryffindor)
X = df[['Best Hand', 'Age', 'Herbology', 'Defense Against the Dark Arts',
        'Potions', 'Charms', 'Flying']]

# 3. Agregar columna de 1's para el término de sesgo (bias)
X = np.c_[np.ones(len(X)), X]

# Convertir a arrays de numpy para operaciones más eficientes
X = np.array(X)
y = np.array(y)

# Las dimensiones deberían ser:
# X: (1277, 8) - 1277 muestras, 8 características (incluyendo el bias)
# y: (1277,) - 1277 etiquetas

Después de ejecutar el código anterior:
- X será una matriz donde la primera columna es toda de 1's (para el bias) y las demás columnas son nuestras características
- y será un vector con nuestras etiquetas (0 o 1 para Gryffindor)

Los datos ya están normalizados, así que no necesitamos hacer ninguna normalización adicional

Para comprobar las dimensiones de las matrices puedes usar el atributo shape de NumPy.

In [5]:
# Comprobar dimensiones
print("Dimensiones de X:", X.shape)
print("Dimensiones de y:", y.shape)

# También podemos ver las primeras filas para verificar que la estructura es correcta
print("\nPrimeras 3 filas de X (mostrando el término de bias en la primera columna):")
headers = ['Bias', 'Best Hand', 'Age', 'Herbology', 'Defense Against the Dark Arts',
           'Potions', 'Charms', 'Flying']

# Crear una tabla con las primeras 3 filas de X
table = tabulate(X[:3], headers=headers, floatfmt='.6f', tablefmt='fancy_grid')
print(table)

print("\nPrimeros 3 valores de y:")
print(y[:3])

Dimensiones de X: (1277, 8)
Dimensiones de y: (1277,)

Primeras 3 filas de X (mostrando el término de bias en la primera columna):
╒══════════╤═════════════╤═══════════╤═════════════╤═════════════════════════════════╤═══════════╤═══════════╤═══════════╕
│     Bias │   Best Hand │       Age │   Herbology │   Defense Against the Dark Arts │   Potions │    Charms │    Flying │
╞══════════╪═════════════╪═══════════╪═════════════╪═════════════════════════════════╪═══════════╪═══════════╪═══════════╡
│ 1.000000 │    0.000000 │ -0.630834 │    0.866628 │                        1.021499 │ -0.702829 │  1.197913 │ -0.506096 │
├──────────┼─────────────┼───────────┼─────────────┼─────────────────────────────────┼───────────┼───────────┼───────────┤
│ 1.000000 │    1.000000 │ -0.307800 │   -1.376018 │                        1.144493 │  0.412213 │ -1.010371 │ -1.393587 │
├──────────┼─────────────┼───────────┼─────────────┼─────────────────────────────────┼───────────┼───────────┼───────────┤
│ 1.0000

Al correr el código anterior la salida debería ser algo como:
- Para X: (1277, 8) - indicando 1277 filas y 8 columnas
- Para y: (1277,) - indicando un vector de 1277 elementos

En la visualización de las primeras filas de X, deberíamos ver que la primera columna es toda de 1's (el término de bias que agregamos).

## Función sigmoide
Vamos a implementar la función sigmoide.  
Esta función es crucial en la regresión logística ya que transforma cualquier número real en un valor entre 0 y 1, que podemos interpretar como una probabilidad.

Esta es la función sigmoide

σ(z) = 1/(1 + e^(-z))

luego sustituiremos z por una función lineal con una serie de parámetros cuyos valores debemos estimar y que esos valores serán los pesos de nuestro modelo entrenado.

Si la función z es:

$$z = θ_0 + θ_1 \cdot x_1 + θ_2 \cdot x_2 + \cdots + θ_n \cdot x_n$$

El valor de $n$ debe coincidir con el número de características de nuestro modelo, en este caso hay 7 características, por lo que $n=7$ puesto que se cumple que $θₙ = θ₇$

Cuando preparamos los datos, añadimos una columna extra de 1's para el término de sesgo (bias). Así que ahora X tiene 8 columnas:
- La primera columna de 1's (para θ₀, el término de sesgo)
- Las 7 características originales

Entonces la función z completa sería:

z = θ₀ * 1 + θ₁ * x₁ + θ₂ * x₂ + θ₃ * x₃ + θ₄ * x₄ + θ₅ * x₅ + θ₆ * x₆ + θ₇ * x₇

Donde:
- θ₀ es el término de sesgo (bias) que multiplica a la columna de 1's que añadimos
- θ₁ hasta θ₇ son los pesos que multiplican a nuestras 7 características originales

Esto se puede escribir de forma más compacta como una multiplicación matricial:

$$z = X \cdot θ$$

Donde:
- $X$ es nuestra matriz de (1277, 8)
- $θ$ será un vector de 8 parámetros que debemos estimar
- $z$ será un vector de 1277 valores que luego pasaremos por la función sigmoide

In [6]:
def sigmoid(z):
    """
    Calcula la función sigmoide: σ(z) = 1/(1 + e^(-z))

    Parámetros:
    z: puede ser un número real, vector o matriz

    Retorna:
    Valor de la función sigmoide
    """
    # Usamos np.clip para evitar desbordamiento numérico
    # Limitamos los valores a [-250, 250] para evitar warnings de overflow
    z_safe = np.clip(z, -250, 250)
    return 1.0 / (1.0 + np.exp(-z_safe))

# Podemos probar la función con algunos valores para verificar que funciona correctamente
test_values = np.array([-10, -1, 0, 1, 10])
print("Valores de prueba:", test_values)
print("Valores sigmoide:", sigmoid(test_values))

Valores de prueba: [-10  -1   0   1  10]
Valores sigmoide: [4.53978687e-05 2.68941421e-01 5.00000000e-01 7.31058579e-01
 9.99954602e-01]


El código anterior:
- Utiliza NumPy para manejar operaciones vectorizadas
- Incluye protección contra desbordamiento numérico usando np.clip
- Puede manejar tanto valores individuales como arrays

Al ejecutar este código, deberíamos ver que:
- Para valores muy negativos, la función se acerca a 0
- Para z = 0, la función da exactamente 0.5
- Para valores muy positivos, la función se acerca a 1

## Función de pérdida
Vamos a implementar la función de pérdida logarítmica (log loss) para regresión logística.
Para cada observación, la función de pérdida es:
- Si y = 1: -log(h(x))
- Si y = 0: -log(1 - h(x))

Donde h(x) es nuestra predicción (la salida de la función sigmoide).

Esto se puede escribir de forma compacta para todo el conjunto de datos como:
$$J(θ) = -(1/m) * Σ [y * log(h(x)) + (1-y) * log(1-h(x))]$$
Donde:
- m es el número de observaciones (1277 en nuestro caso)
- y son los valores reales
- h(x) son las predicciones (después de aplicar la sigmoide)
- Σ representa la suma sobre todas las observaciones

In [7]:
def compute_cost(X, y, theta):
    """
    Calcula la función de pérdida logarítmica

    Parámetros:
    X: matriz de características (incluyendo columna de 1's)
    y: vector de etiquetas reales
    theta: vector de parámetros

    Retorna:
    J: valor de la función de pérdida
    """
    m = len(y)

    # Calcular predicciones
    z = np.dot(X, theta)
    h = sigmoid(z)

    # Calcular pérdida logarítmica
    # Añadimos un pequeño valor epsilon para evitar log(0)
    epsilon = 1e-15
    J = -(1/m) * np.sum(y * np.log(h + epsilon) + (1-y) * np.log(1 - h + epsilon))

    return J

El código anterior:
- Calcula z = X·θ
- Aplica la función sigmoide para obtener h(x)
- Calcula la pérdida logarítmica
- Incluye un pequeño valor epsilon para evitar problemas numéricos con log(0)

## Cálculo del Gradiente
Para el cálculo del gradiente, necesitamos derivar la función de pérdida con respecto a cada parámetro θⱼ.

En regresión logística, el gradiente tiene una forma muy elegante:
∂J/∂θ = (1/m) * X^T * (h(x) - y)

Donde:
- m es el número de observaciones (1277)
- X^T es la matriz X transpuesta
- h(x) - y es la diferencia entre nuestras predicciones y los valores reales



In [8]:
def compute_gradient(X, y, theta):
    """
    Calcula el gradiente de la función de pérdida

    Parámetros:
    X: matriz de características (incluyendo columna de 1's)
    y: vector de etiquetas reales
    theta: vector de parámetros actual

    Retorna:
    gradient: vector con las derivadas parciales respecto a cada parámetro
    """
    m = len(y)

    # Calcular predicciones
    z = np.dot(X, theta)
    h = sigmoid(z)

    # Calcular gradiente
    gradient = (1/m) * np.dot(X.T, (h - y))

    return gradient

El código anterior:
- Calcula las predicciones actuales usando los parámetros theta
- Calcula el error (h - y)
- Multiplica por X transpuesta y normaliza por m

El gradiente resultante tendrá 8 componentes (una por cada parámetro θ), que nos indicarán en qué dirección debemos actualizar cada parámetro para minimizar la función de pérdida.

## Descenso del Gradiente
Implementaremos el algoritmo de descenso del gradiente que usará todas las funciones que hemos creado anteriormente.

Early stopping: detener el algoritmo cuando el cambio en el coste sea menor que un umbral (por ejemplo, 1e-8)

Para implementear el Early stopping:
- Agregamos el parámetro epsilon para el umbral
- Comparamos el coste actual con el anterior
- Si la diferencia es menor que epsilon, detenemos el algoritmo
- Informamos en qué iteración se alcanzó la convergencia

Esto debería reducir significativamente el tiempo de ejecución al evitar iteraciones innecesarias una vez que el modelo haya convergido.

In [11]:
def gradient_descent(X, y, learning_rate=0.1, num_iterations=1000, epsilon=1e-8):
    """
    Implementa el descenso del gradiente con early stopping

    Parámetros:
    X: matriz de características (incluyendo columna de 1's)
    y: vector de etiquetas reales
    learning_rate: tasa de aprendizaje (alpha)
    num_iterations: número máximo de iteraciones
    epsilon: umbral para early stopping

    Retorna:
    theta: parámetros optimizados
    cost_history: lista con el valor de la función de pérdida en cada iteración
    """
    # Inicializar parámetros theta con ceros
    theta = np.zeros(X.shape[1])

    # Lista para guardar el historial de costes
    cost_history = []

    # Calcular coste inicial
    prev_cost = compute_cost(X, y, theta)
    cost_history.append(prev_cost)

    # Descenso del gradiente
    for i in range(num_iterations):
        # Calcular gradiente y actualizar parámetros
        gradient = compute_gradient(X, y, theta)
        theta = theta - learning_rate * gradient

        # Calcular nuevo coste
        current_cost = compute_cost(X, y, theta)
        cost_history.append(current_cost)

        # Imprimir progreso cada 100 iteraciones
        if i % 1000 == 0:
            print(f'Iteración {i}: Coste = {current_cost}')

        # Early stopping
        if abs(prev_cost - current_cost) < epsilon:
            print(f'\nConvergencia alcanzada en la iteración {i}')
            print(f'Diferencia en coste: {abs(prev_cost - current_cost)}')
            break

        prev_cost = current_cost

    return theta, cost_history

Este algoritmo:
1. Inicializa los parámetros θ con ceros
2. En cada iteración:
    - Calcula el gradiente actual
    - Actualiza los parámetros usando la fórmula: θ = θ - α * gradiente
    - Guarda el valor de la función de pérdida
3. Retorna los parámetros optimizados y el historial de costes

Podemos ejecutarlo así:

In [12]:
# Hiperparámetros
learning_rate = 0.1
num_iterations = 100_000
# Ejecutar el descenso del gradiente
theta_optimal, cost_history = gradient_descent(X, y, learning_rate, num_iterations)

Iteración 0: Coste = 0.6537569229648154
Iteración 1000: Coste = 0.051618788591932106
Iteración 2000: Coste = 0.04746862053989086
Iteración 3000: Coste = 0.04576635291997387
Iteración 4000: Coste = 0.044801627076967165
Iteración 5000: Coste = 0.04417646848250042
Iteración 6000: Coste = 0.04373418874275613
Iteración 7000: Coste = 0.04340134660257361
Iteración 8000: Coste = 0.043139329631335877
Iteración 9000: Coste = 0.04292596845806905
Iteración 10000: Coste = 0.04274764423387861
Iteración 11000: Coste = 0.04259553490739291
Iteración 12000: Coste = 0.04246367470089986
Iteración 13000: Coste = 0.04234788192144806
Iteración 14000: Coste = 0.042245133928530465
Iteración 15000: Coste = 0.0421531863852637
Iteración 16000: Coste = 0.04207033263037326
Iteración 17000: Coste = 0.04199524677885572
Iteración 18000: Coste = 0.041926878627245714
Iteración 19000: Coste = 0.041864381587421215
Iteración 20000: Coste = 0.04180706223448379
Iteración 21000: Coste = 0.041754344325962804
Iteración 22000: C

## Accuracy
Aquí si usaremos la librería `sklearn` para calcular la precisión del model.

Vamos a calcular la accuracy del modelo. Para esto, necesitamos:
- Hacer predicciones usando nuestros parámetros theta optimizados
- Convertir las probabilidades en clases (0 o 1)
- Comparar con los valores reales usando sklearn.metrics

In [16]:
from sklearn.metrics import accuracy_score

def predict(X, theta):
    """
    Realiza predicciones usando los parámetros optimizados

    Parámetros:
    X: matriz de características
    theta: parámetros optimizados

    Retorna:
    y_pred: predicciones (0 o 1)
    """
    # Calcular probabilidades
    z = np.dot(X, theta)
    probabilities = sigmoid(z)

    # Convertir a clases (0 o 1)
    predictions = (probabilities >= 0.5).astype(int)

    return predictions

# Hacer predicciones
y_pred = predict(X, theta_optimal)

# Calcular accuracy
accuracy = accuracy_score(y, y_pred)
print(f'Accuracy del modelo: {accuracy:.4f}')

Accuracy del modelo: 0.9922


El código anterior:
- Define una función predict que:
    - Calcula z = X·θ
    - Aplica la función sigmoide
    - Convierte probabilidades a clases usando 0.5 como umbral

- Usa los parámetros theta_optimal que obtuvimos del descenso del gradiente
- Calcula la accuracy usando sklearn.metrics

### ¿Qué es la Accuracy?
La Accuracy (precisión o exactitud) es una de las métricas más intuitivas para evaluar modelos de clasificación. Se basa en un concepto simple pero poderoso:

Accuracy = (Número de predicciones correctas) / (Número total de predicciones)

En el contexto de clasificación binaria (como nuestro caso con Gryffindor/No Gryffindor), podemos desglosarlo así:

- Verdaderos Positivos (TP): Predijimos Gryffindor y era Gryffindor
- Verdaderos Negativos (TN): Predijimos No-Gryffindor y era No-Gryffindor
- Falsos Positivos (FP): Predijimos Gryffindor pero era No-Gryffindor
- Falsos Negativos (FN): Predijimos No-Gryffindor pero era Gryffindor

La fórmula completa sería:
Accuracy = (TP + TN) / (TP + TN + FP + FN)

Es importante notar que:
- La Accuracy es útil cuando las clases están balanceadas
- No es tan informativa cuando hay desbalance de clases
- Un valor de 0.5 en clasificación binaria equivale a predicción aleatoria
- Un valor de 1.0 significa predicción perfecta

Dos ejemplos:
- si nuestro modelo predice correctamente la casa de 1000 estudiantes de un total de 1277, la accuracy sería 1000/1277 ≈ 0.783 o 78.3%.
- si nuestro modelo predice correctamente la casa de 1267 estudiantes de un total de 1277, la accuracy sería 1267/1277 ≈ 0.992169146436962 o **99,22%**. En este caso solo se han clasificado incorrectamente 10 casos de un total de 1277.